# Import

In [1]:
!cp -r ../input/python-packages2 ./

!tar xvfz ./python-packages2/jiwer.tgz
!pip install ./jiwer/jiwer-2.3.0-py3-none-any.whl -f ./ --no-index
!tar xvfz ./python-packages2/normalizer.tgz
!pip install ./normalizer/bnunicodenormalizer-0.0.24.tar.gz -f ./ --no-index
!tar xvfz ./python-packages2/pyctcdecode.tgz
!pip install ./pyctcdecode/attrs-22.1.0-py2.py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/exceptiongroup-1.0.0rc9-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/hypothesis-6.54.4-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/pygtrie-2.5.0.tar.gz -f ./ --no-index --no-deps
!pip install ./pyctcdecode/sortedcontainers-2.4.0-py2.py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/pyctcdecode-0.4.0-py2.py3-none-any.whl -f ./ --no-index --no-deps

!tar xvfz ./python-packages2/pypikenlm.tgz
!pip install ./pypikenlm/pypi-kenlm-0.1.20220713.tar.gz -f ./ --no-index --no-deps

import os
!python -m pip install --no-index --find-links=../input/bengaliai-pip-wheels-demucs demucs

jiwer/
jiwer/jiwer-2.3.0-py3-none-any.whl
jiwer/python-Levenshtein-0.12.2.tar.gz
jiwer/setuptools-65.3.0-py3-none-any.whl
Looking in links: ./
Processing ./jiwer/jiwer-2.3.0-py3-none-any.whl
INFO: pip is looking at multiple versions of jiwer to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement python-Levenshtein==0.12.2 (from jiwer) (from versions: none)
ERROR: No matching distribution found for python-Levenshtein==0.12.2
normalizer/
normalizer/bnunicodenormalizer-0.0.24.tar.gz
Looking in links: ./
Processing ./normalizer/bnunicodenormalizer-0.0.24.tar.gz
  Preparing metadata (setup.py) ... - done
  Created wheel for bnunicodenormalizer: filename=bnunicodenormalizer-0.0.24-py3-none-any.whl size=17628 sha256=34e0ca1d54a5895c72a046e250e10068089a38b1f23a525b162f939a6f7b99f8
  Stored in directory: /root/.cache/pip/wheels/78/d7/75/6986dc3616718f950b80e3bd79a796ef618eaef6cd800e7909
Successf

In [2]:
rm -r python-packages2 jiwer normalizer pyctcdecode pypikenlm

In [3]:
!mkdir ./generated_audio

# Demucs

In [4]:
%%python
import glob
import subprocess
import os

import typing as tp
from pathlib import Path
from functools import partial
from dataclasses import dataclass, field

import pandas as pd
import pyctcdecode
import numpy as np
from tqdm.notebook import tqdm

import librosa

import pyctcdecode
import kenlm
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ProcessorWithLM, Wav2Vec2ForCTC
from bnunicodenormalizer import Normalizer

import cloudpickle as cpkl

ROOT = Path.cwd().parent
INPUT = ROOT / "input"
DATA = INPUT / "bengaliai-speech"
TRAIN = DATA / "train_mp3s"
TEST = DATA / "test_mp3s"

SAMPLING_RATE = 16_000
MODEL_PATH = INPUT / "bengali-sr-download-public-trained-models/indicwav2vec_v1_bengali/"
LM_PATH = INPUT / "bengali-sr-download-public-trained-models/wav2vec2-xls-r-300m-bengali/language_model/"

# Customize the following options!
model = "htdemucs"
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
# two_stems = None   # only separate one stems from the rest, for instance
two_stems = "vocals"

# Options for the output audio.
mp3 = True
mp3_rate = 48000
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True` !!

in_path = TEST
out_path = './music_output'
repo_path = '/kaggle/input/demucs-model'

#@title Useful functions, don't forget to execute
import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model, "--repo", repo_path]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")

separate()

dirlist = glob.glob(f'./{out_path}/htdemucs/*/')
for d in dirlist:
    before = os.path.join(d, 'vocals.mp3')
    after = os.path.join('./generated_audio', d[:-1].replace(f'./{out_path}/htdemucs/', '')+'.mp3')
    subprocess.run(f"mv {before} {after}",
                   shell=True, capture_output=True, text=True)

TEST = Path('./generated_audio')

# import soundfile as sf
for d in glob.glob(str(TEST/'*')):
    audio = librosa.load(d, sr=16000)[0]

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Going to separate the files:
/kaggle/input/bengaliai-speech/test_mp3s/a9395e01ad21.mp3
/kaggle/input/bengaliai-speech/test_mp3s/0f3dac00655e.mp3
/kaggle/input/bengaliai-speech/test_mp3s/bf36ea8b718d.mp3
With command:  python3 -m demucs.separate -o ./music_output -n htdemucs --repo /kaggle/input/demucs-model --mp3 --mp3-bitrate=48000 --two-stems=vocals
Separated tracks will be stored in /kaggle/working/music_output/htdemucs
Separating track /kaggle/input/bengaliai-speech/test_mp3s/a9395e01ad21.mp3


100%|██████████████████████████████████████████████████████████████████████████| 11.7/11.7 [00:05<00:00,  2.32seconds/s]


Separating track /kaggle/input/bengaliai-speech/test_mp3s/0f3dac00655e.mp3


100%|██████████████████████████████████████████████████████████████████████████| 5.85/5.85 [00:00<00:00, 34.22seconds/s]


Separating track /kaggle/input/bengaliai-speech/test_mp3s/bf36ea8b718d.mp3


100%|██████████████████████████████████████████████████████████████████████████| 5.85/5.85 [00:00<00:00, 34.29seconds/s]


# Inference1

In [5]:
%%python
import typing as tp
from pathlib import Path
from functools import partial
from dataclasses import dataclass, field

import re
import pandas as pd
import pyctcdecode
import numpy as np
from tqdm.notebook import tqdm

import librosa

import pyctcdecode
import kenlm
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ProcessorWithLM, Wav2Vec2ForCTC
from bnunicodenormalizer import Normalizer

import cloudpickle as cpkl

ROOT = Path.cwd().parent
INPUT = ROOT / "input"
DATA = INPUT / "bengaliai-speech"
TRAIN = DATA / "train_mp3s"
TEST = DATA / "test_mp3s"

SAMPLING_RATE = 16_000
MODEL_PATH = INPUT / "bengali-sr-download-public-trained-models/indicwav2vec_v1_bengali/"
LM_PATH = INPUT / "bengali-sr-download-public-trained-models/wav2vec2-xls-r-300m-bengali/language_model/"


if not torch.cuda.is_available():
    device = torch.device("cpu")
else:
    device = torch.device("cuda")

model = Wav2Vec2ForCTC.from_pretrained(MODEL_PATH)
processor = Wav2Vec2Processor.from_pretrained(MODEL_PATH)

model.load_state_dict(torch.load('/kaggle/input/baisr-ver5-3/model_ver5_3_WER0.32805.pth', map_location=device)['model'])

vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

decoder = pyctcdecode.build_ctcdecoder(
    list(sorted_vocab_dict.keys()),
    str("/kaggle/input/5gramlm4/5gram_normalized_bn_train_commonvoice_fleurs_openslr_openslr37_oscar_without_punc_prune2.arpa"),
)

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

class BengaliSRTestDataset(torch.utils.data.Dataset):
    
    def __init__(
        self,
        audio_paths: list[str],
        sampling_rate: int
    ):
        self.audio_paths = audio_paths
        self.sampling_rate = sampling_rate
        
    def __len__(self,):
        return len(self.audio_paths)
    
    def __getitem__(self, index: int):
        audio_path = self.audio_paths[index]
        sr = self.sampling_rate
        w = librosa.load(audio_path, sr=sr)[0]

        return w

test = pd.read_csv(DATA / "sample_submission.csv", dtype={"id": str})
test['original_index'] = test.index
test['path'] = [str(TEST / f"{aid}.mp3") for aid in test["id"].values]

def read_audio(mp3_path, target_sr=16000):
    audio, sr = librosa.load(mp3_path, sr=32000)
    return audio

def get_audio_length(row):
    audio = read_audio(row['path'])
    row['audio_length'] = len(audio)/32000 
    return row

test = test.apply(get_audio_length, axis=1)

test = test.sort_values('audio_length').reset_index(drop=True)

test_audio_paths = [str(TEST / f"{aid}.mp3") for aid in test["id"].values]

test_dataset = BengaliSRTestDataset(
    test_audio_paths, SAMPLING_RATE
)

collate_func = partial(
    processor_with_lm.feature_extractor,
    return_tensors="pt", sampling_rate=SAMPLING_RATE,
    padding=True,
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=8, shuffle=False,
    num_workers=2, collate_fn=collate_func, drop_last=False,
    pin_memory=True,
)

model = model.to(device)
model = model.eval()
model = model.half()

pred_sentence_list = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        x = batch["input_values"]
        x = x.to(device, non_blocking=True)
        with torch.cuda.amp.autocast(True):
            y = model(x).logits
        y = y.detach().cpu().numpy()
        
        for l in y:  
            sentence = processor_with_lm.decode(l, beam_width=10).text
            pred_sentence_list.append(sentence)

bnorm = Normalizer()

def postprocess(sentence):
    _words = [bnorm(word)['normalized']  for word in sentence.split()]
    sentence = " ".join([word for word in _words if word is not None])
    try:
        sentence = " ".join(re.sub('[\,\।\?\!\-]', " ", sentence).split())
        if len(sentence) == 0:
            sentence = "।"
    except:
        sentence = "।"
    return sentence

pp_pred_sentence_list = [
    postprocess(s) for s in tqdm(pred_sentence_list)]

test["sentence"] = pp_pred_sentence_list
test = test.sort_values('original_index').reset_index(drop=True).drop(['original_index', 'audio_length', 'path'], axis=1)
test.to_csv("pre_submission_no_demucs.csv", index=False)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

  0%|          | 0/3 [00:00<?, ?it/s]


In [6]:
%%python
import typing as tp
from pathlib import Path
from functools import partial
from dataclasses import dataclass, field

import re
import pandas as pd
import pyctcdecode
import numpy as np
from tqdm.notebook import tqdm

import librosa

import pyctcdecode
import kenlm
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ProcessorWithLM, Wav2Vec2ForCTC
from bnunicodenormalizer import Normalizer

import cloudpickle as cpkl

ROOT = Path.cwd().parent
INPUT = ROOT / "input"
DATA = INPUT / "bengaliai-speech"
TRAIN = DATA / "train_mp3s"
TEST = Path('./generated_audio')

SAMPLING_RATE = 16_000
MODEL_PATH = INPUT / "bengali-sr-download-public-trained-models/indicwav2vec_v1_bengali/"
LM_PATH = INPUT / "bengali-sr-download-public-trained-models/wav2vec2-xls-r-300m-bengali/language_model/"


if not torch.cuda.is_available():
    device = torch.device("cpu")
else:
    device = torch.device("cuda")

model = Wav2Vec2ForCTC.from_pretrained(MODEL_PATH)
processor = Wav2Vec2Processor.from_pretrained(MODEL_PATH)

model.load_state_dict(torch.load('/kaggle/input/baisr-ver5-3/model_ver5_3_WER0.32805.pth', map_location=device)['model'])

vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

decoder = pyctcdecode.build_ctcdecoder(
    list(sorted_vocab_dict.keys()),
    str("/kaggle/input/5gramlm4/5gram_normalized_bn_train_commonvoice_fleurs_openslr_openslr37_oscar_without_punc_prune2.arpa"),
)

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

class BengaliSRTestDataset(torch.utils.data.Dataset):
    
    def __init__(
        self,
        audio_paths: list[str],
        sampling_rate: int
    ):
        self.audio_paths = audio_paths
        self.sampling_rate = sampling_rate
        
    def __len__(self,):
        return len(self.audio_paths)
    
    def __getitem__(self, index: int):
        audio_path = self.audio_paths[index]
        sr = self.sampling_rate
        w = librosa.load(audio_path, sr=sr)[0]

        return w

test = pd.read_csv(DATA / "sample_submission.csv", dtype={"id": str})
test['original_index'] = test.index
test['path'] = [str(TEST / f"{aid}.mp3") for aid in test["id"].values]

def read_audio(mp3_path, target_sr=16000):
    audio, sr = librosa.load(mp3_path, sr=32000)
    return audio

def get_audio_length(row):
    audio = read_audio(row['path'])
    row['audio_length'] = len(audio)/32000 
    return row

test = test.apply(get_audio_length, axis=1)

test = test.sort_values('audio_length').reset_index(drop=True)

test_audio_paths = [str(TEST / f"{aid}.mp3") for aid in test["id"].values]

test_dataset = BengaliSRTestDataset(
    test_audio_paths, SAMPLING_RATE
)

collate_func = partial(
    processor_with_lm.feature_extractor,
    return_tensors="pt", sampling_rate=SAMPLING_RATE,
    padding=True,
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=8, shuffle=False,
    num_workers=2, collate_fn=collate_func, drop_last=False,
    pin_memory=True,
)

model = model.to(device)
model = model.eval()
model = model.half()

pred_sentence_list = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        x = batch["input_values"]
        x = x.to(device, non_blocking=True)
        with torch.cuda.amp.autocast(True):
            y = model(x).logits
        y = y.detach().cpu().numpy()
        
        for l in y:  
            sentence = processor_with_lm.decode(l, beam_width=10).text
            pred_sentence_list.append(sentence)

bnorm = Normalizer()

def postprocess(sentence):
    _words = [bnorm(word)['normalized']  for word in sentence.split()]
    sentence = " ".join([word for word in _words if word is not None])
    try:
        sentence = " ".join(re.sub('[\,\।\?\!\-]', " ", sentence).split())
        if len(sentence) == 0:
            sentence = "।"
    except:
        sentence = "।"
    return sentence

pp_pred_sentence_list = [
    postprocess(s) for s in tqdm(pred_sentence_list)]

test["sentence"] = pp_pred_sentence_list
test = test.sort_values('original_index').reset_index(drop=True).drop(['original_index', 'audio_length', 'path'], axis=1)
test.to_csv("pre_submission_demucs.csv", index=False)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

  0%|          | 0/3 [00:00<?, ?it/s]


In [7]:
%%python
import pandas as pd
import numpy as np
import subprocess

df_before = pd.read_csv('pre_submission_no_demucs.csv')
df_after = pd.read_csv('pre_submission_demucs.csv')
pred = []
for id, before, after in zip(df_after["id"].values, df_before["sentence"].values, df_after["sentence"].values):
    if len(after.split()) < len(before.split()):
        subprocess.run(f'cp /kaggle/input/bengaliai-speech/test_mp3s/{id}.mp3 ./generated_audio/{id}.mp3', shell=True)

# Inference2

In [8]:
%%python
import typing as tp
from pathlib import Path
from functools import partial
from dataclasses import dataclass, field

import re
import pandas as pd
import pyctcdecode
import numpy as np
from tqdm.notebook import tqdm

import librosa

import pyctcdecode
import kenlm
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ProcessorWithLM, Wav2Vec2ForCTC
from bnunicodenormalizer import Normalizer

import cloudpickle as cpkl

ROOT = Path.cwd().parent
INPUT = ROOT / "input"
DATA = INPUT / "bengaliai-speech"
TRAIN = DATA / "train_mp3s"
TEST = Path('./generated_audio')

SAMPLING_RATE = 16_000
MODEL_PATH = INPUT / "bengali-sr-download-public-trained-models/indicwav2vec_v1_bengali/"
LM_PATH = INPUT / "bengali-sr-download-public-trained-models/wav2vec2-xls-r-300m-bengali/language_model/"


if not torch.cuda.is_available():
    device = torch.device("cpu")
else:
    device = torch.device("cuda")

model = Wav2Vec2ForCTC.from_pretrained(MODEL_PATH)
processor = Wav2Vec2Processor.from_pretrained(MODEL_PATH)

model.load_state_dict(torch.load('/kaggle/input/baisr-ver5-3/model_ver5_3_WER0.32805.pth', map_location=device)['model'])

vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

decoder = pyctcdecode.build_ctcdecoder(
    list(sorted_vocab_dict.keys()),
    str("/kaggle/input/5gramlm4/5gram_normalized_bn_train_commonvoice_fleurs_openslr_openslr37_oscar_without_punc_prune2.arpa"),
)

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

class BengaliSRTestDataset(torch.utils.data.Dataset):
    
    def __init__(
        self,
        audio_paths: list[str],
        sampling_rate: int
    ):
        self.audio_paths = audio_paths
        self.sampling_rate = sampling_rate
        
    def __len__(self,):
        return len(self.audio_paths)
    
    def __getitem__(self, index: int):
        audio_path = self.audio_paths[index]
        sr = self.sampling_rate
        w = librosa.load(audio_path, sr=sr)[0]

        return w

test = pd.read_csv(DATA / "sample_submission.csv", dtype={"id": str})
test['original_index'] = test.index
test['path'] = [str(TEST / f"{aid}.mp3") for aid in test["id"].values]

def read_audio(mp3_path, target_sr=16000):
    audio, sr = librosa.load(mp3_path, sr=32000)
    return audio

def get_audio_length(row):
    audio = read_audio(row['path'])
    row['audio_length'] = len(audio)/32000 
    return row

test = test.apply(get_audio_length, axis=1)

test = test.sort_values('audio_length').reset_index(drop=True)

test_audio_paths = [str(TEST / f"{aid}.mp3") for aid in test["id"].values]

test_dataset = BengaliSRTestDataset(
    test_audio_paths, SAMPLING_RATE
)

collate_func = partial(
    processor_with_lm.feature_extractor,
    return_tensors="pt", sampling_rate=SAMPLING_RATE,
    padding=True,
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=8, shuffle=False,
    num_workers=2, collate_fn=collate_func, drop_last=False,
    pin_memory=True,
)

model = model.to(device)
model = model.eval()
model = model.half()

pred_sentence_list = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        x = batch["input_values"]
        x = x.to(device, non_blocking=True)
        with torch.cuda.amp.autocast(True):
            y = model(x).logits
        y = y.detach().cpu().numpy()
        
        for l in y:  
            sentence = processor_with_lm.decode(l, beam_width=256*3).text
            pred_sentence_list.append(sentence)

bnorm = Normalizer()

def postprocess(sentence):
    _words = [bnorm(word)['normalized']  for word in sentence.split()]
    sentence = " ".join([word for word in _words if word is not None])
    try:
        sentence = " ".join(re.sub('[\,\।\?\!\-]', " ", sentence).split())
        if len(sentence) == 0:
            sentence = "।"
    except:
        sentence = "।"
    return sentence

pp_pred_sentence_list = [
    postprocess(s) for s in tqdm(pred_sentence_list)]

test["sentence"] = pp_pred_sentence_list
test = test.sort_values('original_index').reset_index(drop=True).drop(['original_index', 'audio_length', 'path'], axis=1)
test.to_csv("pre_submission.csv", index=False)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

  0%|          | 0/3 [00:00<?, ?it/s]


# Punctuation

In [9]:
import os
os.system('python -m pip install --no-index --find-links=../input/vakyansh-models-punctuation-models-bengali indic-nlp-library')
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
import numpy as np
import pandas as pd
import json
import torch.nn as nn
from indicnlp.tokenize import indic_tokenize

CHECKPOINT_PATH = '/kaggle/input/punctuation-models/xlm-roberta-large_exp016_cp175.pt'
LABEL_ENCODER_PATH = '/kaggle/input/punctuation-models/label_encoder.json'

label_encoder_path = LABEL_ENCODER_PATH
punctuation_dict = {'qm': '? ', 'comma': ', ', 'end': '। ', 'blank': ' ', 'hyp': '-', 'PAD': ' '}

with open(label_encoder_path) as label_encoder:
    train_encoder = json.load(label_encoder)

tokenizer = AutoTokenizer.from_pretrained(
    '/kaggle/input/xlm-roberta-large',
)
model = AutoModelForTokenClassification.from_pretrained(
    '/kaggle/input/xlm-roberta-large',
    num_labels=len(train_encoder),
    output_attentions=False,
    output_hidden_states=False,
)

# model = nn.DataParallel(model)
checkpoint = torch.load(CHECKPOINT_PATH)
model.load_state_dict(checkpoint['state_dict'], strict=False)

model.eval()
model.cuda()

# Added model
CHECKPOINT_PATH_1 = '/kaggle/input/punctuation-models/xlm-roberta-base_exp017_cp200.pt'
tokenizer_1 = AutoTokenizer.from_pretrained(
    '/kaggle/input/bengaliai-xlm-roberta-base',
)
model_1 = AutoModelForTokenClassification.from_pretrained(
    '/kaggle/input/bengaliai-xlm-roberta-base',
    num_labels=len(train_encoder),
    output_attentions=False,
    output_hidden_states=False,
)
checkpoint_1 = torch.load(CHECKPOINT_PATH_1)
model_1.load_state_dict(checkpoint_1['state_dict'], strict=False)
model_1.eval()
model_1.cuda()

def get_tokens_and_labels_indices_from_text(text):
    # MODEL 0
    tokenized_sentence = tokenizer.encode(text)
    input_ids = torch.tensor([tokenized_sentence]).cuda()
    with torch.no_grad():
        output = model(input_ids)
    label_indices = output[0].to('cpu').numpy()
    tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
    # MODEL 1
    tokenized_sentence_1 = tokenizer_1.encode(text)
    input_ids_1 = torch.tensor([tokenized_sentence_1]).cuda()
    with torch.no_grad():
        output_1 = model_1(input_ids_1)
    label_indices_1 = output_1[0].to('cpu').numpy()
    tokens_1 = tokenizer_1.convert_ids_to_tokens(input_ids_1.to('cpu').numpy()[0])
    return tokens, label_indices, tokens_1, label_indices_1

def map_tokens_and_labels_to_word_and_punctuations(text):
    if len(text) > 2048:
        full_text = " ".join(text.split())
        if full_text[-1] not in ["।", "!", "?", "-"]:
            full_text += "।"
        return full_text
    tokens, label_indices, tokens_1, label_indices_1 = get_tokens_and_labels_indices_from_text(text)
    new_tokens = []
    new_labels = []
    for i in range(1, len(tokens) - 1):
        if tokens[i].startswith("▁"):
            current_word = tokens[i][1:]
            new_labels.append(label_indices[0][i])
            for j in range(i + 1, len(tokens) - 1):
                if not tokens[j].startswith("▁"):
                    current_word = current_word + tokens[j]
                if tokens[j].startswith("▁"):
                    break
            new_tokens.append(current_word)
    full_text = ''
    tokenized_text = indic_tokenize.trivial_tokenize_indic(text)
    
    if len(tokenized_text) == len(new_labels):
        full_text_tokens = tokenized_text
    else:
        full_text_tokens = new_tokens
    new_tokens_1 = []
    new_labels_1 = []
    for i in range(1, len(tokens_1) - 1):
        if tokens_1[i].startswith("▁"):
            current_word_1 = tokens_1[i][1:]
            new_labels_1.append(label_indices_1[0][i])
            for j in range(i + 1, len(tokens_1) - 1):
                if not tokens_1[j].startswith("▁"):
                    current_word_1 = current_word_1 + tokens_1[j]
                if tokens_1[j].startswith("▁"):
                    break
            new_tokens_1.append(current_word_1)
    if len(new_labels) == len(new_labels_1):
        for word, punctuation_0, punctuation_1 in zip(full_text_tokens, new_labels, new_labels_1):
            x_0 = np.exp(punctuation_0 - np.max(punctuation_0))
            pred_0 = x_0 / np.sum(x_0)
            x_1 = np.exp(punctuation_1 - np.max(punctuation_1))
            pred_1 = x_1 / np.sum(x_1)
            punctuation = pred_0 * 0.95 + pred_1 * 0.05
            punctuation = np.argmax(punctuation)
            punctuation = list(train_encoder.keys())[list(train_encoder.values()).index(punctuation)]
            full_text = full_text + word + punctuation_dict[punctuation]
    else:
        for word, punctuation in zip(full_text_tokens, new_labels):
            punctuation = np.argmax(punctuation)
            punctuation = list(train_encoder.keys())[list(train_encoder.values()).index(punctuation)]
            full_text = full_text + word + punctuation_dict[punctuation]
    
    full_text = " ".join(full_text.split())
    if len(full_text) == 0:
        full_text = " ".join(text.split())
        if len(full_text) == 0:
            full_text = "।"
    if full_text[-1] in [",", "-"]:
        full_text = full_text[:-1] + "।"
    if full_text[-1] not in ["।", "!", "?"]:
        full_text += "।"
    
    return full_text

sub = pd.read_csv("pre_submission.csv")
sub['sentence'] = sub['sentence'].apply(map_tokens_and_labels_to_word_and_punctuations)

from bnunicodenormalizer import Normalizer
bnorm = Normalizer()
def normalize(sentence):
    word = [bnorm(word)['normalized'] for word in sentence.split()]
    return " ".join([w for w in word if w is not None])
sub['sentence'] = sub['sentence'].apply(lambda x: normalize(x))

Looking in links: ../input/vakyansh-models-punctuation-models-bengali
Processing /kaggle/input/vakyansh-models-punctuation-models-bengali/indic_nlp_library-0.92-py3-none-any.whl
Processing /kaggle/input/vakyansh-models-punctuation-models-bengali/sphinx_argparse-0.4.0-py3-none-any.whl (from indic-nlp-library)
Processing /kaggle/input/vakyansh-models-punctuation-models-bengali/Morfessor-2.0.6-py3-none-any.whl (from indic-nlp-library)
Processing /kaggle/input/vakyansh-models-punctuation-models-bengali/sphinx-7.2.6-py3-none-any.whl (from sphinx-argparse->indic-nlp-library)
Processing /kaggle/input/vakyansh-models-punctuation-models-bengali/sphinxcontrib_applehelp-1.0.7-py3-none-any.whl (from sphinx>=1.2.0->sphinx-argparse->indic-nlp-library)
Processing /kaggle/input/vakyansh-models-punctuation-models-bengali/sphinxcontrib_devhelp-1.0.5-py3-none-any.whl (from sphinx>=1.2.0->sphinx-argparse->indic-nlp-library)
Processing /kaggle/input/vakyansh-models-punctuation-models-bengali/sphinxcontrib_

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [10]:
!rm -rf ./*

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
sub.to_csv("submission.csv", index=False)